In [1]:
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

## 추천받은 포트폴리오 -> 잔고(detail 테이블

In [2]:
instruments_m = pd.read_pickle('../../data/processed/instruments_m.pkl')
price_db = pd.read_pickle('../../data/external/price_db_d.pkl')
advised_pf = pd.read_pickle('../../data/processed/advised_portfolios.pkl')    

In [3]:
userid = 'A00'
username = '투자자0'
current_date = '2021-02-01'
risk_profile = 2

# c_date: 추천 포트폴리오DB에서 사용자가 입력한 날짜와 가장 가까운 날짜.
c_date = advised_pf.loc[advised_pf.date <= current_date, ['date']].max().date
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}
df = advised_pf.loc[(advised_pf.date==c_date) & (advised_pf.risk_profile==risk_profile), :]
remaining_cash = 135786

In [4]:
# 보유수량
detail = pd.DataFrame.from_dict(new_units, orient='index').rename(columns={0:'quantity'})
detail.index.name='itemcode'

# 매입가격
temp = pd.DataFrame.from_dict(prices, orient='index').rename(columns={0:'cost_price'}).drop([1], axis=1)
temp.index.name='itemcode'

# 병합
detail = detail.merge(temp, left_index=True, right_index=True, how='inner')

detail['price'] = 
detail['cost_value'] = detail['quantity']*detail['cost_price']
detail['value'] = detail['quantity']*detail['cost_price']  # 매입가와 평가가격 동일하다고 가정
detail = detail.merge(instruments_m.loc[:, ['itemcode', 'itemname', 'asset_class']], left_on='itemcode', right_on='itemcode', how='left')
detail = detail.reset_index(drop=True)

In [20]:
detail['price'] = detail['cost_price']

In [5]:
df_cash = {
    'itemcode': 'C000001',
    'quantity': remaining_cash,
    'cost_price': 1,
    'cost_value': remaining_cash,
    'value': remaining_cash,
    'itemname': '현금',
    'asset_class': 'Cash'
}
df_cash = pd.DataFrame.from_dict(df_cash, orient='index').T

In [6]:
detail = pd.concat([detail, df_cash])

In [13]:
detail['date'] = c_date
detail['date'] = pd.to_datetime(detail['date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y 4:0:00 PM').astype(str)
detail['userid'] = userid
detail['username'] = username
detail['original'] = 'N'
detail['group_by'] = ''

In [14]:
detail

,itemcode,quantity,cost_price,cost_value,value,itemname,asset_class,date,userid,username,wt,original,group_by
0,A122260,2,101185,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.202421,N,
1,A114260,4,57900,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.231659,N,
2,A130730,2,100975,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.202001,N,
3,A329650,13,10980,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,0.142776,N,
4,A266370,4,21325,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,0.0853217,N,
5,C000001,135786,1,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,0.13582,N,


In [8]:
detail = detail.reset_index(drop=True)

In [9]:
detail

,itemcode,quantity,cost_price,cost_value,value,itemname,asset_class,date,userid,username
0,A122260,2,101185,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0
1,A114260,4,57900,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0
2,A130730,2,100975,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0
3,A329650,13,10980,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0
4,A266370,4,21325,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0
5,C000001,135786,1,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0


In [10]:
detail['wt'] = detail.value.transform(lambda x: x/x.sum())

In [11]:
detail

,itemcode,quantity,cost_price,cost_value,value,itemname,asset_class,date,userid,username,wt
0,A122260,2,101185,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.202421
1,A114260,4,57900,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.231659
2,A130730,2,100975,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.202001
3,A329650,13,10980,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,0.142776
4,A266370,4,21325,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,0.0853217
5,C000001,135786,1,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,0.13582


In [15]:
from datetime import datetime

In [25]:
temp = datetime.strptime('2021-02-01', '%Y-%m-%d').strftime('%m/%d/%Y 4:0:00 PM')

In [31]:
pd.to_datetime(detail['date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y 4:0:00 PM').astype(str)

0    02/01/2021 4:0:00 PM
1    02/01/2021 4:0:00 PM
2    02/01/2021 4:0:00 PM
3    02/01/2021 4:0:00 PM
4    02/01/2021 4:0:00 PM
5    02/01/2021 4:0:00 PM
Name: date, dtype: object

In [16]:
66*4*6*12

19008

In [19]:
detail

,itemcode,quantity,cost_price,cost_value,value,itemname,asset_class,date,userid,username,wt,original,group_by
0,A122260,2,101185,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.202421,N,
1,A114260,4,57900,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.231659,N,
2,A130730,2,100975,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,0.202001,N,
3,A329650,13,10980,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,0.142776,N,
4,A266370,4,21325,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,0.0853217,N,
5,C000001,135786,1,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,0.13582,N,


In [23]:
insert_query_dtl = 'INSERT INTO {0} (itemcode, quantity, cost_price, price, cost_value, value' \
                   'itemname, asset_class, date, userid, username, group_by, original, wt) ' \
                    'values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [24]:
insert_query_dtl.format('detail')

'INSERT INTO detail (itemcode, quantity, cost_price, price, cost_value, valueitemname, asset_class, date, userid, username, group_by, original, wt) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'